In [8]:
def create_cubes(n):
    for i in range(n):
        yield i**3


In [9]:
create_cubes(10)

<generator object create_cubes at 0x118a47d30>

In [10]:
for i in create_cubes(10):
    print(i)

0
1
8
27
64
125
216
343
512
729


In [55]:
import functools
def gen_func(original_function):
    @functools.wraps(original_function)
    def ase():
        print("before original function")
        original_function()
        print("after original_function")
    return ase

In [56]:
@gen_func
def test_func():
    """This is a test func"""
    print("inside test_func()")

In [57]:
print(test_func.__doc__)

This is a test func


In [59]:
test_func()

before original function
inside test_func()
after original_function


In [72]:
class Addstr:
    def __init__(self, cls):
        self.cls = cls

    def __call__(self, *args, **kwargs):

        instance = self.cls(*args, **kwargs)

        def __str__(self_inner):
            return (f"{self.cls.__name__} instance : {self_inner.__dict__}")

        setattr(self.cls, '__str__', __str__)
        return instance

@Addstr
class Myclass:
    def __init__(myself, name, age):
        myself.name = name 
        myself.age = age
            

In [73]:
print(Myclass("vkt",32))

Myclass instance : {'name': 'vkt', 'age': 32}


# Rubrik Interview Question

In [13]:
from threading import Thread
from threading import Condition
import time

class ProducerConsumer:
    def __init__(self, capacity=5):
        self.capacity = capacity
        self.queue = [None] * capacity
        self.head = 0
        self.count = 0

    def enqueue(self, item):
        pos = (self.head + self.count) % self.capacity
        self.queue[pos] = item
        self.count += 1
        self.head = pos
        print(f"Enqueued item {item} Queue : {self.queue}")
        self.simulateDelay()

    def dequeue(self):
        pos = self.head
        item = self.queue[pos]
        self.queue[pos] = None
        self.count -= 1
        self.head = (self.head + 1) % self.capacity
        print(f"Dequeued item {item} Queue : {self.queue}")
        self.simulateDelay()

    def simulateDelay(self):
        time.sleep(0.1)

def main():
    p = ProducerConsumer(5)
    for i in range(8):
        t = Thread(target=p.enqueue, args=(i,))
        t.start()
    for _ in range(8):
        t = Thread(target=p.dequeue)
        t.start()

main()
        

Enqueued item 0 Queue : [0, None, None, None, None]
Enqueued item 1 Queue : [0, 1, None, None, None]
Enqueued item 2 Queue : [0, 1, None, 2, None]
Enqueued item 3 Queue : [0, 3, None, 2, None]
Enqueued item 4 Queue : [4, 3, None, 2, None]
Enqueued item 5 Queue : [5, 3, None, 2, None]
Enqueued item 6 Queue : [5, 6, None, 2, None]
Enqueued item 7 Queue : [5, 6, None, 7, None]
Dequeued item 7 Queue : [5, 6, None, None, None]
Dequeued item None Queue : [5, 6, None, None, None]
Dequeued item 5 Queue : [None, 6, None, None, None]
Dequeued item 6 Queue : [None, None, None, None, None]
Dequeued item None Queue : [None, None, None, None, None]
Dequeued item None Queue : [None, None, None, None, None]
Dequeued item None Queue : [None, None, None, None, None]
Dequeued item None Queue : [None, None, None, None, None]


## Solution of above

In [5]:
from threading import Thread
from threading import Condition
import time

cond = Condition()

class ProducerConsumer:
    def __init__(self, capacity=5):
        self.capacity = capacity
        self.queue = [None] * capacity
        self.head = 0
        self.tail = 0
        self.count = 0
        self.cond = Condition()

    def enqueue(self, item):
        with self.cond:
            while self.count >= self.capacity:
                self.cond.wait()
            pos = (self.head + 1) % self.capacity
            self.queue[pos] = item
            self.count += 1
            self.head = pos
            print(f"Enqueued item {item} Queue : {self.queue}")
            self.simulateDelay()
            self.cond.notify_all()

    def dequeue(self):
        with self.cond:
            while self.count <= 0:
                self.cond.wait()
            pos = self.tail
            item = self.queue[pos]
            self.queue[pos] = None
            self.count -= 1
            self.tail = (self.tail + 1) % self.capacity
            print(f"Dequeued item {item} Queue : {self.queue}")
            self.simulateDelay()
            self.cond.notify_all()

    def simulateDelay(self):
        time.sleep(0.5)

def main():
    p = ProducerConsumer(5)
    threads = []
    for i in range(8):
        t = Thread(target=p.enqueue, args=(i,))
        t.start()
        threads.append(t)
    for _ in range(8):
        t = Thread(target=p.dequeue)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
main()
        


Enqueued item 0 Queue : [None, 0, None, None, None]
Enqueued item 1 Queue : [None, 0, 1, None, None]
Enqueued item 2 Queue : [None, 0, 1, 2, None]
Enqueued item 3 Queue : [None, 0, 1, 2, 3]
Enqueued item 4 Queue : [4, 0, 1, 2, 3]
Dequeued item 4 Queue : [None, 0, 1, 2, 3]
Dequeued item 0 Queue : [None, None, 1, 2, 3]
Dequeued item 1 Queue : [None, None, None, 2, 3]
Dequeued item 2 Queue : [None, None, None, None, 3]
Dequeued item 3 Queue : [None, None, None, None, None]
Enqueued item 5 Queue : [None, 5, None, None, None]
Enqueued item 6 Queue : [None, 5, 6, None, None]
Enqueued item 7 Queue : [None, 5, 6, 7, None]
Dequeued item None Queue : [None, 5, 6, 7, None]
Dequeued item 5 Queue : [None, None, 6, 7, None]
Dequeued item 6 Queue : [None, None, None, 7, None]


In [3]:
from threading import Thread, Lock
import time

class ProducerConsumer:
    def __init__(self, capacity=5):
        self.capacity = capacity
        self.queue = [None] * capacity
        self.head = 0  # Next position to dequeue
        self.tail = 0  # Next position to enqueue
        self.count = 0
        self.lock = Lock()

    def enqueue(self, item):
        while True:
            with self.lock:
                if self.count < self.capacity:
                    self.queue[self.tail] = item
                    self.tail = (self.tail + 1) % self.capacity
                    self.count += 1
                    print(f"[Producer] Enqueued {item} | Queue: {self.queue}")
                    break
            # Sleep a bit before retrying to avoid busy waiting
            time.sleep(0.01)
        self.simulateDelay()

    def dequeue(self):
        while True:
            with self.lock:
                if self.count > 0:
                    item = self.queue[self.head]
                    self.queue[self.head] = None
                    self.head = (self.head + 1) % self.capacity
                    self.count -= 1
                    print(f"[Consumer] Dequeued {item} | Queue: {self.queue}")
                    break
            time.sleep(0.01)
        self.simulateDelay()

    def simulateDelay(self):
        time.sleep(0.1)

def main():
    p = ProducerConsumer(5)

    # Start consumers first
    for _ in range(8):
        Thread(target=p.dequeue).start()

    # Start producers
    for i in range(8):
        Thread(target=p.enqueue, args=(i,)).start()

main()

[Producer] Enqueued 0 | Queue: [0, None, None, None, None]
[Producer] Enqueued 1 | Queue: [0, 1, None, None, None]
[Producer] Enqueued 2 | Queue: [0, 1, 2, None, None]
[Producer] Enqueued 3 | Queue: [0, 1, 2, 3, None]
[Producer] Enqueued 4 | Queue: [0, 1, 2, 3, 4]
[Consumer] Dequeued 0 | Queue: [None, 1, 2, 3, 4]
[Consumer] Dequeued 1 | Queue: [None, None, 2, 3, 4]
[Consumer] Dequeued 2 | Queue: [None, None, None, 3, 4]
[Consumer] Dequeued 3 | Queue: [None, None, None, None, 4]
[Consumer] Dequeued 4 | Queue: [None, None, None, None, None]
[Producer] Enqueued 5 | Queue: [5, None, None, None, None]
[Producer] Enqueued 6 | Queue: [5, 6, None, None, None]
[Producer] Enqueued 7 | Queue: [5, 6, 7, None, None]
[Consumer] Dequeued 5 | Queue: [None, 6, 7, None, None]
[Consumer] Dequeued 6 | Queue: [None, None, 7, None, None]
[Consumer] Dequeued 7 | Queue: [None, None, None, None, None]
